In [39]:
import numpy as np

In [55]:
!cp /Users/gilles/Downloads/stn_surfaces/*.asc /Users/gilles/git/nukkies/notebooks/data

In [171]:
os.chdir('/Users/gilles/git/nukkies/notebooks/')

In [172]:
!ls data

BI3T_STh_L.asc     KP6T_STh_L_dil.asc PF5T_STh_R.asc     VL1T_STh_R_dil.asc
BI3T_STh_L_dil.asc KP6T_STh_R.asc     PF5T_STh_R_dil.asc WSFT_STh_L.asc
BI3T_STh_R.asc     KP6T_STh_R_dil.asc RSIT_STh_L.asc     WSFT_STh_L_dil.asc
BI3T_STh_R_dil.asc LV2T_STh_L.asc     RSIT_STh_L_dil.asc WSFT_STh_R.asc
FMFT_STh_L.asc     LV2T_STh_L_dil.asc RSIT_STh_R.asc     WSFT_STh_R_dil.asc
FMFT_STh_L_dil.asc LV2T_STh_R.asc     RSIT_STh_R_dil.asc WW2T_STh_L.asc
FMFT_STh_R.asc     LV2T_STh_R_dil.asc TS6T_STh_L.asc     WW2T_STh_L_dil.asc
FMFT_STh_R_dil.asc MRCT_STh_L.asc     TS6T_STh_L_dil.asc WW2T_STh_R.asc
HCBT_STh_L.asc     MRCT_STh_L_dil.asc TS6T_STh_R.asc     WW2T_STh_R_dil.asc
HCBT_STh_L_dil.asc MRCT_STh_R.asc     TS6T_STh_R_dil.asc smoothed
HCBT_STh_R.asc     MRCT_STh_R_dil.asc UM2T_STh_L.asc     stn_L.asc
HCBT_STh_R_dil.asc NM3T_STh_L.asc     UM2T_STh_L_dil.asc stn_R.asc
KCAT_STh_L.asc     NM3T_STh_L_dil.asc UM2T_STh_R.asc     test.asc
KCAT_STh_L_dil.asc NM3T_STh_R.asc     UM2T_STh_R_dil.asc test2.asc

In [78]:
!mkdir data/smoothed

In [173]:
import re
path = '/Users/gilles/git/nukkies/notebooks/data'
reg = re.compile('.*/(?P<subject_id>[A-Z0-9]{4})_(?P<mask>[a-zA-Z_]+)\.asc')

In [174]:
fn

'/Users/gilles/git/nukkies/notebooks/data/smoothed/WW2T_STh_R_smooth.asc'

In [175]:
import subprocess

In [176]:
import os

In [190]:
import glob
fns = glob.glob(os.path.join(path, '*_dil.asc'))

for fn in fns:
    new_fn = os.path.join(path, 'smoothed/{subject_id}_{mask}_smooth.asc'.format(**reg.match(fn).groupdict()))
    
    # Due to crappiness of Freesurfer, it does not easily run in a notebook. This code
    # was executed in a terminal
    print('mris_smooth -a 1 -n 1 -nw {fn} {new_fn}'.format(**locals()))

mris_smooth -a 1 -n 1 -nw /Users/gilles/git/nukkies/notebooks/data/BI3T_STh_L_dil.asc /Users/gilles/git/nukkies/notebooks/data/smoothed/BI3T_STh_L_dil_smooth.asc
mris_smooth -a 1 -n 1 -nw /Users/gilles/git/nukkies/notebooks/data/BI3T_STh_R_dil.asc /Users/gilles/git/nukkies/notebooks/data/smoothed/BI3T_STh_R_dil_smooth.asc
mris_smooth -a 1 -n 1 -nw /Users/gilles/git/nukkies/notebooks/data/FMFT_STh_L_dil.asc /Users/gilles/git/nukkies/notebooks/data/smoothed/FMFT_STh_L_dil_smooth.asc
mris_smooth -a 1 -n 1 -nw /Users/gilles/git/nukkies/notebooks/data/FMFT_STh_R_dil.asc /Users/gilles/git/nukkies/notebooks/data/smoothed/FMFT_STh_R_dil_smooth.asc
mris_smooth -a 1 -n 1 -nw /Users/gilles/git/nukkies/notebooks/data/HCBT_STh_L_dil.asc /Users/gilles/git/nukkies/notebooks/data/smoothed/HCBT_STh_L_dil_smooth.asc
mris_smooth -a 1 -n 1 -nw /Users/gilles/git/nukkies/notebooks/data/HCBT_STh_R_dil.asc /Users/gilles/git/nukkies/notebooks/data/smoothed/HCBT_STh_R_dil_smooth.asc
mris_smooth -a 1 -n 1 -nw /U

In [191]:
def load_data(fn, norm=True):
    f = open(fn)
    lines = f.readlines(1000)
    f.close()
    
    n_vertices, n_faces = lines[1].split()
    n_vertices, n_faces = int(n_vertices), int(n_faces)

    vertices = np.genfromtxt(fn, skip_header=2, skip_footer=n_faces, usecols=[0,1, 2])
    faces = np.genfromtxt(fn, skip_header=2 + n_vertices, usecols=[0,1, 2])
    
    if norm:
        vertices -= vertices.mean(0)
        vertices /= (vertices.max(0) - vertices.min(0))

    return vertices, faces

In [193]:
fns = glob.glob(os.path.join(path, 'smoothed', '*_dil_smooth.asc'))

for fn in fns:
    print(fn)
    vertices, faces = load_data(fn)
    
    
    
    new_fn = '/Users/gilles/git/nukkies/data/{subject_id}_{mask}.obj'.format(**reg.match(fn).groupdict())

    f = open(new_fn, 'w')
    f.writelines('o {subject_id}_{mask}\n'.format(**reg.match(fn).groupdict()))
    f.close()
    
    with open(new_fn, 'ab') as f:
        np.savetxt(f, vertices, 'v %f %f %f', )

    with open(new_fn, 'ab') as f:
        np.savetxt(f, faces+1, 'f %d %d %d')
    

/Users/gilles/git/nukkies/notebooks/data/smoothed/BI3T_STh_L_dil_smooth.asc
/Users/gilles/git/nukkies/notebooks/data/smoothed/BI3T_STh_R_dil_smooth.asc
/Users/gilles/git/nukkies/notebooks/data/smoothed/FMFT_STh_L_dil_smooth.asc
/Users/gilles/git/nukkies/notebooks/data/smoothed/FMFT_STh_R_dil_smooth.asc
/Users/gilles/git/nukkies/notebooks/data/smoothed/HCBT_STh_L_dil_smooth.asc
/Users/gilles/git/nukkies/notebooks/data/smoothed/HCBT_STh_R_dil_smooth.asc
/Users/gilles/git/nukkies/notebooks/data/smoothed/KCAT_STh_L_dil_smooth.asc
/Users/gilles/git/nukkies/notebooks/data/smoothed/KCAT_STh_R_dil_smooth.asc
/Users/gilles/git/nukkies/notebooks/data/smoothed/KP6T_STh_L_dil_smooth.asc
/Users/gilles/git/nukkies/notebooks/data/smoothed/KP6T_STh_R_dil_smooth.asc
/Users/gilles/git/nukkies/notebooks/data/smoothed/LV2T_STh_L_dil_smooth.asc
/Users/gilles/git/nukkies/notebooks/data/smoothed/LV2T_STh_R_dil_smooth.asc
/Users/gilles/git/nukkies/notebooks/data/smoothed/MRCT_STh_L_dil_smooth.asc
/Users/gille

In [188]:
!ls /Users/gilles/git/nukkies/data/

BI3T_STh_L_dil_smooth.obj LV2T_STh_L_dil_smooth.obj TS6T_STh_L_dil_smooth.obj
BI3T_STh_R_dil_smooth.obj LV2T_STh_R_dil_smooth.obj TS6T_STh_R_dil_smooth.obj
FMFT_STh_L_dil_smooth.obj MRCT_STh_L_dil_smooth.obj UM2T_STh_L_dil_smooth.obj
FMFT_STh_R_dil_smooth.obj MRCT_STh_R_dil_smooth.obj UM2T_STh_R_dil_smooth.obj
HCBT_STh_L_dil_smooth.obj NM3T_STh_L_dil_smooth.obj VL1T_STh_L_dil_smooth.obj
HCBT_STh_R_dil_smooth.obj NM3T_STh_R_dil_smooth.obj VL1T_STh_R_dil_smooth.obj
KCAT_STh_L_dil_smooth.obj PF5T_STh_L_dil_smooth.obj WSFT_STh_L_dil_smooth.obj
KCAT_STh_R_dil_smooth.obj PF5T_STh_R_dil_smooth.obj WSFT_STh_R_dil_smooth.obj
KP6T_STh_L_dil_smooth.obj RSIT_STh_L_dil_smooth.obj WW2T_STh_L_dil_smooth.obj
KP6T_STh_R_dil_smooth.obj RSIT_STh_R_dil_smooth.obj WW2T_STh_R_dil_smooth.obj


Make some javascript "code"

In [189]:
os.chdir('/Users/gilles/git/nukkies/')
fns = glob.glob('data/*.obj')

for fn in fns:
    print("'{fn}': null,".format(**locals()))

'data/BI3T_STh_L_dil_smooth.obj': null,
'data/BI3T_STh_R_dil_smooth.obj': null,
'data/FMFT_STh_L_dil_smooth.obj': null,
'data/FMFT_STh_R_dil_smooth.obj': null,
'data/HCBT_STh_L_dil_smooth.obj': null,
'data/HCBT_STh_R_dil_smooth.obj': null,
'data/KCAT_STh_L_dil_smooth.obj': null,
'data/KCAT_STh_R_dil_smooth.obj': null,
'data/KP6T_STh_L_dil_smooth.obj': null,
'data/KP6T_STh_R_dil_smooth.obj': null,
'data/LV2T_STh_L_dil_smooth.obj': null,
'data/LV2T_STh_R_dil_smooth.obj': null,
'data/MRCT_STh_L_dil_smooth.obj': null,
'data/MRCT_STh_R_dil_smooth.obj': null,
'data/NM3T_STh_L_dil_smooth.obj': null,
'data/NM3T_STh_R_dil_smooth.obj': null,
'data/PF5T_STh_L_dil_smooth.obj': null,
'data/PF5T_STh_R_dil_smooth.obj': null,
'data/RSIT_STh_L_dil_smooth.obj': null,
'data/RSIT_STh_R_dil_smooth.obj': null,
'data/TS6T_STh_L_dil_smooth.obj': null,
'data/TS6T_STh_R_dil_smooth.obj': null,
'data/UM2T_STh_L_dil_smooth.obj': null,
'data/UM2T_STh_R_dil_smooth.obj': null,
'data/VL1T_STh_L_dil_smooth.obj': null,


In [165]:
from scipy import ndimage

In [ ]:
ndimage.binary_closing('')